In [9]:
pip install delta-spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824013 sha256=4838fdf9acc385df86af377462eb64c2187aa4c5af394f9ad1c23ff2ab92a8b5
  Stored in directory: /home/jovyan/.cache/pip/wheels/89/d6/52/1178e354ba2207673484f0ccd7b2ded0ab6671ae5c1fc5b49a
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.2.0
    Can't uninstall 'pyspark'. No files were found to uninstall.
Note: you may need to restart the kernel to use updated packages.


In [1]:
### Load data into postgres database - staging schema 

In [1]:
################################################################################
import os
import sys
import pyspark
from pyspark.sql import SparkSession
from delta.tables import *
from delta.tables import DeltaTable
from pyspark.sql.functions import *
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql import functions as f
from pyspark.sql.functions import from_unixtime, col, to_timestamp
from pyspark.sql.functions import udf
import hashlib
import urllib.request
import json
from datetime import timedelta, date
from pyspark import SparkContext
from pyspark import SQLContext
from itertools import islice
from pyspark.sql.functions import col
################################################################################

In [2]:
################################################################################
import os
import sys
import pyspark
from pyspark.sql import SparkSession
from delta.tables import *
from delta.tables import DeltaTable
from pyspark.sql.functions import *
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql import functions as f
from pyspark.sql.functions import from_unixtime, col, to_timestamp
from pyspark.sql.functions import udf
import hashlib
import urllib.request
import json
from datetime import timedelta, date
from pyspark import SparkContext
from pyspark import SQLContext
from itertools import islice
from pyspark.sql.functions import col
################################################################################

In [3]:
# Spark session & context
spark = (SparkSession
         .builder
         .master("local")
         .appName("read-postgres")
         # Add postgres jar
         .config("spark.driver.extraClassPath", "/home/jovyan/work/jars/postgresql-42.4.0.jar")
         .getOrCreate())
sc = spark.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/24 15:14:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/24 15:14:28 WARN SparkSession: Cannot use io.delta.sql.DeltaSparkSessionExtension to configure session extensions.
java.lang.ClassNotFoundException: io.delta.sql.DeltaSparkSessionExtension
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:589)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:522)
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(Class.java:398)
	at org.apache.spark.util.Utils$.classForName(Utils.scala:216)
	at org.apache.spark.sql.SparkSession$.$anonfun$applyExtensions$1(SparkS

In [6]:
# Define schema for orders data
orders_schema = StructType([
    StructField("customer_id", IntegerType(), False),
    StructField("order_date", DateType(), False),
    StructField("product_id", IntegerType(), False),
    StructField("unit_price", IntegerType(), False),
    StructField("quantity", IntegerType(), False),
    StructField("total_price", IntegerType(), False)
])

# Load orders data from CSV file into DataFrame
orders_df = spark.read.format("csv") \
    .option("header", True) \
    .option("inferSchema", False) \
    .schema(orders_schema) \
    .load("/home/jovyan/work/data/orders.csv")

In [7]:
orders_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- unit_price: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- total_price: integer (nullable = true)



In [10]:
(orders_df.write
 .format("jdbc")
 .option("url", "jdbc:postgresql://oasispostgres:5432/metastore")
 .option("dbtable", "public.orders")
 .option("user", "hive")
 .option("password", "hive")
 .mode("overwrite")
 .save())

Py4JJavaError: An error occurred while calling o83.save.
: org.apache.spark.SparkException: Cannot find catalog plugin class for catalog 'spark_catalog': org.apache.spark.sql.delta.catalog.DeltaCatalog
	at org.apache.spark.sql.errors.QueryExecutionErrors$.catalogPluginClassNotFoundForCatalogError(QueryExecutionErrors.scala:1454)
	at org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:64)
	at org.apache.spark.sql.connector.catalog.CatalogManager.loadV2SessionCatalog(CatalogManager.scala:66)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$v2SessionCatalog$2(CatalogManager.scala:85)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$v2SessionCatalog$1(CatalogManager.scala:85)
	at scala.Option.map(Option.scala:230)
	at org.apache.spark.sql.connector.catalog.CatalogManager.v2SessionCatalog(CatalogManager.scala:84)
	at org.apache.spark.sql.connector.catalog.CatalogManage

In [ ]:
# PostgreSQL database credentials
jdbc_url = "jdbc:postgresql://d2b-internal-assessment-dwh.cxeuj0ektqdz.eu-central-1.rds.amazonaws.com:5432/d2b_assessment"
db_properties = {
    "user": "mahmoyin9767",
    "password": "yDXXxChDDH",
    "driver": "org.postgresql.Driver"
}

In [ ]:
# Write orders DataFrame to PostgreSQL database table
orders_df.write \
    .mode("overwrite") \
    .option("truncate", "true") \
    .option("numPartitions", "10") \
    .jdbc(jdbc_url, table="mahmoyin9767_staging.orders_staging", properties=db_properties)

# Close SparkSession

In [3]:
import psycopg2
from psycopg2 import sql
import csv

# PostgreSQL database credentials
host = 'd2b-internal-assessment-dwh.cxeuj0ektqdz.eu-central-1.rds.amazonaws.com'
port = '5432'
database = 'd2b_assessment'
user = 'mahmoyin9767'
password = 'yDXXxChDDH'

# Connection to PostgreSQL database
conn = psycopg2.connect(
    host=host,
    port=port,
    dbname=database,
    user=user,
    password=password
)

# Create orders staging table
with conn.cursor() as cur:
    cur.execute("""
        DROP TABLE IF EXISTS mahmoyin9767_staging.orders_staging;
        CREATE TABLE mahmoyin9767_staging.orders_staging(
            order_id SERIAL PRIMARY KEY,
            customer_id INTEGER NOT NULL,
            order_date DATE NOT NULL,
            product_id INTEGER NOT NULL,
            unit_price INTEGER NOT NULL,
            quantity INTEGER NOT NULL,
            total_price INTEGER NOT NULL
        );
    """)
    conn.commit()

# Load orders data from CSV file and insert into orders staging table
with open('/home/jovyan/work/data/orders.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader) # Skip header row
    with conn.cursor() as cur:
        for row in reader:
            cur.execute(sql.SQL("""
                INSERT INTO mahmoyin9767_staging.orders_staging(
                    customer_id, order_date, product_id, unit_price, quantity, total_price
                ) VALUES ({}, {}, {}, {}, {}, {});
            """).format(
                sql.Literal(int(row[0])), # customer_id
                sql.Literal(row[1]),      # order_date
                sql.Literal(int(row[2])), # product_id
                sql.Literal(int(row[3])), # unit_price
                sql.Literal(int(row[4])), # quantity
                sql.Literal(int(row[5]))  # total_price
            ))
        conn.commit()

# Close database connection
conn.close()


ModuleNotFoundError: No module named 'psycopg2'